# Imports

In [4]:
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [17]:
from src.data import movielens_mf_dataloaders
from src.hyperparams_tuning import tune_params, final_train

# MovieLens 1M

## Data Loaders

In [15]:
dl_ml_train, dl_ml_valid, dl_ml_test, dl_ml_full_train = movielens_mf_dataloaders(batch_size=128)

Movie-lens file already exists


## HyperParameter Tuning

In [16]:
study_ml, df_tuning_results_ml = tune_params(
    model_name='MF', 
    dataset_name='movielens', 
    n_trials=2, 
    dl_train=dl_ml_train, 
    dl_valid=dl_ml_valid, 
    device=device
    )

[I 2022-01-30 19:26:01,120] A new study created in memory with name: no-name-75a9bbf1-e25e-4690-8a41-07fe14ae690d


Training Results - Epoch[1] Avg loss: 1.11
Validation Results - Epoch[1] Avg loss: 1.35
Training Results - Epoch[2] Avg loss: 1.05
Validation Results - Epoch[2] Avg loss: 1.22
Training Results - Epoch[3] Avg loss: 1.02
Validation Results - Epoch[3] Avg loss: 1.18
Training Results - Epoch[4] Avg loss: 1.00
Validation Results - Epoch[4] Avg loss: 1.16
Training Results - Epoch[5] Avg loss: 0.99
Validation Results - Epoch[5] Avg loss: 1.18
Training Results - Epoch[6] Avg loss: 0.98
Validation Results - Epoch[6] Avg loss: 1.18
Training Results - Epoch[7] Avg loss: 0.97
Validation Results - Epoch[7] Avg loss: 1.15
Training Results - Epoch[8] Avg loss: 0.97
Validation Results - Epoch[8] Avg loss: 1.14
Training Results - Epoch[9] Avg loss: 0.96
Validation Results - Epoch[9] Avg loss: 1.14
Training Results - Epoch[10] Avg loss: 0.96


[I 2022-01-30 19:30:42,884] Trial 0 finished with value: 1.1367224906503959 and parameters: {'learning_rate': 0.013488590079462676, 'optimizer': 'RMSprop', 'latent_dim': 16}. Best is trial 0 with value: 1.1367224906503959.


Validation Results - Epoch[10] Avg loss: 1.14
Training Results - Epoch[1] Avg loss: 1.02
Validation Results - Epoch[1] Avg loss: 1.16
Training Results - Epoch[2] Avg loss: 0.97
Validation Results - Epoch[2] Avg loss: 1.12
Training Results - Epoch[3] Avg loss: 0.96
Validation Results - Epoch[3] Avg loss: 1.09
Training Results - Epoch[4] Avg loss: 0.95
Validation Results - Epoch[4] Avg loss: 1.08
Training Results - Epoch[5] Avg loss: 0.94
Validation Results - Epoch[5] Avg loss: 1.07
Training Results - Epoch[6] Avg loss: 0.93
Validation Results - Epoch[6] Avg loss: 1.08
Training Results - Epoch[7] Avg loss: 0.92
Validation Results - Epoch[7] Avg loss: 1.07
Training Results - Epoch[8] Avg loss: 0.92
Validation Results - Epoch[8] Avg loss: 1.08
Training Results - Epoch[9] Avg loss: 0.91
Validation Results - Epoch[9] Avg loss: 1.06
Training Results - Epoch[10] Avg loss: 0.91


[I 2022-01-30 19:37:03,056] Trial 1 finished with value: 1.0591665750038821 and parameters: {'learning_rate': 0.010825257093666968, 'optimizer': 'Adam', 'latent_dim': 11}. Best is trial 1 with value: 1.0591665750038821.


Validation Results - Epoch[10] Avg loss: 1.06


In [19]:
best_params_ml = study_ml.best_params
df_tuning_results_ml.sort_values(by='value').head(15)

,number,value,datetime_start,datetime_complete,duration,params_latent_dim,params_learning_rate,params_optimizer,state
1,1,1.059167,2022-01-30 19:30:42.885798,2022-01-30 19:37:03.056120,0 days 00:06:20.170322,11,0.010825,Adam,COMPLETE
0,0,1.136722,2022-01-30 19:26:01.128533,2022-01-30 19:30:42.884390,0 days 00:04:41.755857,16,0.013489,RMSprop,COMPLETE


## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [20]:
test_loss_ml, final_model_ml, test_mrr_ml = final_train(
    model_name='MF', 
    dataset_name='movielens', 
    params=best_params_ml, 
    dl_full_train=dl_ml_full_train, 
    dl_test=dl_ml_test, 
    device=device
    )
print(f'Final test loss = {test_loss_ml}\nFinal test MRR = {test_mrr_ml}')

Training Results - Epoch[1] Avg loss: 1.03
Validation Results - Epoch[1] Avg loss: 1.15
Training Results - Epoch[2] Avg loss: 0.99
Validation Results - Epoch[2] Avg loss: 1.11
Training Results - Epoch[3] Avg loss: 0.96
Validation Results - Epoch[3] Avg loss: 1.10
Training Results - Epoch[4] Avg loss: 0.95
Validation Results - Epoch[4] Avg loss: 1.12
Training Results - Epoch[5] Avg loss: 0.94
Validation Results - Epoch[5] Avg loss: 1.08
Training Results - Epoch[6] Avg loss: 0.92
Validation Results - Epoch[6] Avg loss: 1.09
Training Results - Epoch[7] Avg loss: 0.92
Validation Results - Epoch[7] Avg loss: 1.10
Training Results - Epoch[8] Avg loss: 0.92
Validation Results - Epoch[8] Avg loss: 1.06
Training Results - Epoch[9] Avg loss: 0.91
Validation Results - Epoch[9] Avg loss: 1.06
Training Results - Epoch[10] Avg loss: 0.91
Validation Results - Epoch[10] Avg loss: 1.07
Final test loss = 1.0598941291215707


# Netflix

In [27]:
%reload_ext tensorboard
%tensorboard --logdir=.

Reusing TensorBoard on port 6006 (pid 25193), started 9:27:25 ago. (Use '!kill 25193' to kill it.)

In [25]:
%load_ext tensorboard
%tensorboard --logdir=.

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 25193), started 9:27:17 ago. (Use '!kill 25193' to kill it.)